## ВКР: обработка данных

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

На этом этапе мы предобработаем наши данные, собранные на этапе `scraping.ipynb`.

По итогу мы получим признаки:
- ррр
- ррр
See Project Proposal.

Где библиотеки??? See `scraping.ipynb`

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit
import re
import numpy as np

In [2]:
import nltk
# nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
morph = MorphAnalyzer()

### Пост-обработка

Импорт данных:

In [3]:
# df = pd.read_csv('resumes_features.csv')
df = pd.read_csv('backup/16.11/resumes_features.csv').sample(3000)

**Перевод опыта работы в числовое значение:**

В данной функции мы приводим к единому виду опыт работы. были найдены: years, year, months, etc.

In [4]:
df['Опыт работы, лет'][2]
opyt_normalized = []
for x in df['Опыт работы, лет']:
    x=x.removeprefix('Work experience ').removeprefix('Опыт работы ')
    x=x.replace('years', ';')
    x=x.replace('year', ';')
    x=x.replace('года', ';')  # ВСЕГДА д. б. раньше
    x=x.replace('год', ';')
    x=x.replace('лет', ';')
    
    if x.find(';') == -1:
        x = '0;' + x
    x=x.removesuffix('months')
    x=x.removesuffix('month')
    x=x.removesuffix('месяцев')
    x=x.removesuffix('месяц')
    x=x.removesuffix('месяца')
     
    x=x.replace(' ', '')
    if re.search('[а–Я]+|[a–Z]+', x):
        x='---'
    opyt_normalized.append(x)
    
df['Опыт нормализованный'] = opyt_normalized
df.drop(list(df)[0], axis=1)

,Unnamed: 0.1,Ссылка,Название,Коммандировка,"Опыт работы, лет",О себе,Образование,Интересы,Навыки,Образование-1,...,Опыт нормализованный,Топовость образования,is_eng?,Уровень оборазования нормализованный,Навыки списком,Релокация?,Коммандировка?,Город?,Аналитические комбинации слов,Справочник по интересам
0,0,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...,Analyst,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",Higher education\n2015\nMOSCOW STATE UNIVERSIT...,"Specializations:\nSales manager, account manag...",Key skills\nAnalitical thinking\nEnglish\nOrga...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,...,7;4,False,ENG,Высшее образование,"['Key skills', 'Analitical thinking', 'English...",prepared for occasional business trips,willing to relocate,Moscow,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,---
1,1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...",Ключевые навыки\nTableau\nPower BI\nSQL\nMS Ex...,"Национальный исследовательский университет ""Вы...",...,1;5,True,RU,Бакалавр,"['Ключевые навыки', 'Tableau', 'Power BI', 'SQ...",готова к редким командировкам,не готова к переезду,Москва,---,found smt by rule (Analyst)|(Аналитик данных)|...
2,2,https://hh.ru/resume/dfbcae000004342d8f0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к команди...",Опыт работы 3 года 3 месяца,---,Высшее образование (Бакалавр)\n2019\nСамарский...,"Специализации:\nBI-аналитик, аналитик данных\n...",Ключевые навыки\nPython\nSQL\nУмение работать ...,Самарский государственный аэрокосмический унив...,...,3;3,False,RU,Бакалавр,"['Ключевые навыки', 'Python', 'SQL', 'Умение р...",готова к командировкам,не готова к переезду,Москва,---,found smt by rule (Analyst)|(Аналитик данных)|...
3,3,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,Аналитик,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,Высшее образование (Бакалавр)\n2015\nСанкт-Пет...,Специализации:\nАналитик\nЗанятость: полная за...,Ключевые навыки\nЭкономический анализ\nЭкономи...,Санкт-Петербургский государственный политехнич...,...,7;2,False,RU,Бакалавр,"['Ключевые навыки', 'Экономический анализ', 'Э...",готов к командировкам,готов к переезду,Санкт,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,found smt by rule (Analyst)|(Аналитик данных)|...
4,4,https://hh.ru/resume/8606f37b000773de980039ed1...,Программист-разработчик,"Москва, м. Петровско-Разумовская, не готов к п...",Опыт работы 1 год 1 месяц,Имеется опыт в создании Android-приложений на ...,Высшее образование (Бакалавр)\n2022\nЧелябинск...,"Специализации:\nПрограммист, разработчик\nЗаня...",Ключевые навыки\nАдминистрирование серверов Li...,"Челябинский государственный университет, Челяб...",...,1;1,False,ENG,Бакалавр,"['Ключевые навыки', 'Администрирование серверо...",не готов к командировкам,не готов к переезду,Москва,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,https://hh.ru/resume/9024b1760002f0f0fe0039ed1...,Business analyst,"Moscow, metro station Akademicheskaya, willing...",Work experience 5 years 5 months,An interest in investment and economics led my...,Higher education (bachelor)\n2017\nBrunel univ...,Specializations:\nAnalyst\nEmployment: full ti...,Key skills\nAnalytical skills\nSAP PM\nBusines...,Brunel university,...,5;5,False,ENG,Бакалавр,"['Key skills', 'Analytical skills', 'SAP PM', ...",NaN,willing to relocate,Moscow,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,found smt by rule (Analyst)|(Аналитик данных)|...
194,194,https://hh.ru/resume/c0d0ac9b0002b4446d0039ed1...,Аналитик,"Москв

**Словарь образования:**

В данной функции мы составляем словарь образования для создания юинарного признака: входит/не входит в топ? Извлечение данной сущности будет самым _неидеальным_ (омонимия аббревиатур вузов етц), но даже погрешность тут будет скомпенсирована большим количеством более простых признаков.

In [7]:
edu_list = pd.concat([df['Образование-1'], df['Образование-2']], axis=0).drop_duplicates().reset_index().drop('index', axis=1)
edu_list = list(edu_list[list(edu_list)[0]])
len(edu_list)

ys=[]

for x in edu_list:
    s=str(x)
    l = s.find(',')+1
    y=s[l:].lstrip(' ')
    print(end='')
    ys.append(y)
    
for i in range(len(ys)):
    y = ys[i]
    is_space = re.fullmatch('[А-я]+[ ][А-я ()-]+', y)
    if is_space:
        #print(y)
        1==1
    is_m = re.fullmatch('Mosc.*', y) or re.fullmatch('Моск.*', y) or re.fullmatch('Петерб.*', y) or re.fullmatch('Peters.*', y) or re.fullmatch('СПб.*', y) or re.fullmatch('Новосиб.*', y) or re.fullmatch('Novosib.*', y) or re.fullmatch('Томск.*', y) or re.fullmatch('Tomsk.*', y)
    if not is_space and not is_m:
        ys[i] = 'NO'
#ys

cnt=0
for y in ys:
    if y != 'NO':
        cnt+=1
cnt

import numpy as np
ys = np.array(ys)
ys=np.unique(ys)
ys=list(ys)
len(ys)

"""
Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

1. Московский государственный университет им. М. В. Ломоносова;
2. Университет ИТМО;
3. Московский физико-технологический институт (МФТИ);
4. Высшая школа экономики;
5. Санкт-Петербургский политехнический университет Петра Великого; 
6. Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;
7. Санкт-Петербургский политехнический университет Петра Великого; 
8. Новосибирский государственный университет;
9. Московский инженерно-физический институт (МИФИ);
10. Томский политехнический университет.

https://propostuplenie.ru/article/spisok-luchshih-informacionnyh-vuzov-rossii/
"""

def is_top(y):
    if re.search('ВШЭ', y) or re.search('Высшая школа экономики', y) or re.search('HSE', y):
        y='ВШЭ'
    elif re.search('Московский Государственный Университет$', y) or re.search('Московский Государственный Университет им[.  ени] (М.|Л)', y) or re.search('МГУ[^ А-я]|\(МГУ\)', y) or re.search('Moscow state University', y) or re.search('MSU', y) or re.search('МГУ', y):
        y='МГУ'
    elif re.search('ИТМО', y):
        y='ИТМО'
    elif re.search('МФТИ', y) or re.search('Московский физико-технологический институт', y):
        y='Физтех'
        # ЗФТШ при МФТИ
        # - Физтех
    elif re.search('СПБ Политех', y) or re.search('Санкт-Петербургский политехнический университет', y):
        y='Политех СПб'
    elif re.search('Бауман', y):
        y='Бауманка'
    elif re.search('НГУ', y) or re.search('Новосибирский государственный университет', y):
        y='НГУ'
        # НГУ Наталии Нестеровой
    elif re.search('МИФИ', y) or re.search('Московский инженерно-физический институт', y):
        y='МИФИ'
        # Какие-то Техникум ДИТИ НИЯУ МИФИ и прочее...
    elif re.search('ТомГУ', y) or re.search('Томский политехнический университет', y):
        y='ТомГУ'
    else:
        y='NO'
    return y != 'NO'
        
# Дальше завожу колонку: parsed_uni по этому правилу -- 1 если хотя бы 1 уник топовый
# И там заменяю значения и вывожу равно ли МГУ / етц.

xs = []
for x in df.index:
    stroke = df.iloc[x].loc[['Образование-1', 'Образование-2']]
    k2 = False
    if type(stroke[0]) != float:
        k1 = is_top(stroke[0])
    if type(stroke[1]) != float:
        k2 = is_top(stroke[1])
    xs.append(k1 or k2)
    
df['Топовость образования']=xs

**Флаг, что текст на eng:**

Здесь мы определим, написан ли текст на английском языке. Сделаем это по разделу "О себе". Понятно, что если он на английском, то и все резюме на английском, поэтому было выбрано именно это поле. 

Старая версия:

```
df['О себе']
cnt=0

for y in df['О себе']:
    if type(y)!=float:  # not nan
        if re.search('[a–Z]+', y):  # более точно: у английских резюме интересы тоже будут на англ итд, потом подумаю над этим
            cnt+=1
cnt, df.shape[0]-cnt

ts=[]
for x in df['О себе']:
    t = 'NOT ENG, NOT RU'
    if type(x)!=float:  # not nan
        t = re.search('[a–Z]+', x)
        if t:
            t = 'ENG'
        else:
            t = 'RU'
    ts.append(t)
df['is_eng?'] = ts
```

In [9]:
ts = []

for i in range(df.shape[0]):
    if type(df['Город?'][i])!=float and type(df['Релокация?'][i])!=float and type(df['Коммандировка?'][i])!=float:  # not nan\
        t = re.search('[A-z]+', df['Город?'][i]) or re.search('[A-z]+', df['Релокация?'][i]) or re.search('[A-z]+', df['Коммандировка?'][i])
        if t:
            t = 'ENG'
        else:
            t = 'RU'

    # print(x, t)
    else:
        t = '---'
    ts.append(t)
        
df['is_eng?'] = ts

In [10]:
about = df['О себе'].loc[df['is_eng?'] == 'RU']

In [11]:
df = df.loc[df['is_eng?'] == 'RU']

In [12]:
about

1      В последние годы проходила обучение без возмож...
2                                                    ---
3      Имею экономическое образование. Продвинутый по...
4      Имеется опыт в создании Android-приложений на ...
5      Уверенный пользователь.\nЖелание оптимизации и...
                             ...                        
192    Интересно автоматизирование процесса.\nИнтерес...
194    Проходил иностранное обучение в Kaplan Interna...
195    Стек технологий:\n[Имеется опыт]: Excel, Oracl...
196    - Обладаю аналитическим складом ума, способен ...
197    В резюме отражены последние четыре места работ...
Name: О себе, Length: 176, dtype: object

In [13]:
df

,Unnamed: 0,Unnamed: 0.1,Ссылка,Название,Коммандировка,"Опыт работы, лет",О себе,Образование,Интересы,Навыки,...,Опыт нормализованный,Топовость образования,is_eng?,Уровень оборазования нормализованный,Навыки списком,Релокация?,Коммандировка?,Город?,Аналитические комбинации слов,Справочник по интересам
1,1,1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...",Ключевые навыки\nTableau\nPower BI\nSQL\nMS Ex...,...,1;5,True,RU,Бакалавр,"['Ключевые навыки', 'Tableau', 'Power BI', 'SQ...",готова к редким командировкам,не готова к переезду,Москва,---,found smt by rule (Analyst)|(Аналитик данных)|...
2,2,2,https://hh.ru/resume/dfbcae000004342d8f0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к команди...",Опыт работы 3 года 3 месяца,---,Высшее образование (Бакалавр)\n2019\nСамарский...,"Специализации:\nBI-аналитик, аналитик данных\n...",Ключевые навыки\nPython\nSQL\nУмение работать ...,...,3;3,False,RU,Бакалавр,"['Ключевые навыки', 'Python', 'SQL', 'Умение р...",готова к командировкам,не готова к переезду,Москва,---,found smt by rule (Analyst)|(Аналитик данных)|...
3,3,3,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,Аналитик,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,Высшее образование (Бакалавр)\n2015\nСанкт-Пет...,Специализации:\nАналитик\nЗанятость: полная за...,Ключевые навыки\nЭкономический анализ\nЭкономи...,...,7;2,False,RU,Бакалавр,"['Ключевые навыки', 'Экономический анализ', 'Э...",готов к командировкам,готов к переезду,Санкт,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,found smt by rule (Analyst)|(Аналитик данных)|...
4,4,4,https://hh.ru/resume/8606f37b000773de980039ed1...,Программист-разработчик,"Москва, м. Петровско-Разумовская, не готов к п...",Опыт работы 1 год 1 месяц,Имеется опыт в создании Android-приложений на ...,Высшее образование (Бакалавр)\n2022\nЧелябинск...,"Специализации:\nПрограммист, разработчик\nЗаня...",Ключевые навыки\nАдминистрирование серверов Li...,...,1;1,False,RU,Бакалавр,"['Ключевые навыки', 'Администрирование серверо...",не готов к командировкам,не готов к переезду,Москва,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,---
5,5,5,https://hh.ru/resume/993bd0980002dd6cff0039ed1...,Аналитик Bi,"Москва, м. Сокол, готов к переезду, готов к ко...",Опыт работы 10 лет 7 месяцев,Уверенный пользователь.\nЖелание оптимизации и...,Высшее образование\n2010\nДальневосточный госу...,"Специализации:\nАналитик\nBI-аналитик, аналити...",Ключевые навыки\nБюджетирование\nЭкономический...,...,10;7,False,RU,Высшее образование,"['Ключевые навыки', 'Бюджетирование', 'Экономи...",готов к командировкам,готов к переезду,Москва,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,found smt by rule (Analyst)|(Аналитик данных)|...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,192,192,https://hh.ru/resume/1a2eedab0009325a2d0039ed1...,Бизнес-аналитик,"Москва, м. Октябрьское поле, не готова к перее...",Опыт работы 3 года 10 месяцев,Интересно автоматизирование процесса.\nИнтерес...,Высшее образование\n2014\nИвановский государст...,Специализации:\nАналитик\nБизнес-аналитик\nЗан...,Ключевые навыки\n1С: Предприятие\nBPMN\nIDEF0\...,...,3;10,False,RU,Высшее образование,"['Ключевые навыки', '1С: Предприятие', 'BPMN',...",готова к командировкам,не готова к переезду,Москва,found smt by rule (SQL)|(Excel)|(Python)|(Tabl...,found smt by rule (Analyst)|(Аналитик данных)|...
194,194,194,https://hh.ru/resume/c0d0ac9b0002b4446d0039ed1...,Аналитик,"Москва, м. Планерная, не готов к переезду, не ...",Опыт работы 5 лет,Проходил иностранное обучение в Kaplan Interna...,Высшее образование\n2015\nРоссийский экономиче...,Специализации:\nАналитик\nЗанятость: стажиров

**Справочник по интересам:**

Есть ли аналитические комбинации слов в интересах? Используем простое правило для простоты.

In [14]:
inter_list = []

for inter in df['Интересы']:
    intr = inter.split('\n')
    if re.search('(Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)', inter):
        inter_list.append('found smt by rule (Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)')
    else:
        inter_list.append('---')
        
df['Справочник по интересам']= inter_list

**Извлечение сущностей:**

На сайте hh.ru единое поле для трех объектов сразу. Разъединим их с учетом разных знаяений и того, что они перечислены просто через запятую.

In [25]:
xs1 = []
xs2=[]
xs3=[]
for x in df['Коммандировка']:
    # english
    x1 = re.search('[A-z]+', x)
    x2 = re.search('[a-z ]*willing to relocate', x)
    x3 = re.search('[a-z ]*prepared for [a-z]* business trips', x)
    if x1:
        x1=x1[0]
    if x2:
        x2=x2[0]
    if x3:
        x3=x3[0]
    
    if x1 or x2 or x3:  # if found smt in engish — is it not wrong way to do it?
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)
    else:
        # russian — work in progress
        # Москва, не готова к переезду, готова к редким командировкам
        # Санкт-Петербург, м. Гражданский проспект, готов к переезду (Москва), готов к командировкам
        # Москва, готов к переезду (Нидерланды, Испания, Франция, Италия, Австрия, Германия, Швейцария), готов к командировкам
        # Москва, хочу переехать, готов к командировкам
        # Москва, м. Ясенево, не готов к переезду, готов к редким командировкам
        x1 = re.search('[А-я]+', x)  # is it correct?
        x2 = re.search('[А-я]*[ не]*готов[а]* к переезду', x)
        x3 = re.search('[А-я]*[ не]*готов[а]* к[ а-я]* командировкам', x)  # черновое правило
        if x1:
            x1=x1[0]
        if x2:
            x2=x2[0]
        if x3:
            x3=x3[0]
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)

zs = []
for z in df['Навыки']:
    zzz = z.split('\n')
    zs.append(zzz)
    
df['Навыки списком'] = zs
df['Релокация?'] = xs3
df['Коммандировка?'] = xs2
df['Город?'] = xs1

"""
1. Коммандировка 
2. Навыки
3. Кол-во работ
4. Наличие аналитических комбинаций слов в работе — новый столбец
"""

import numpy as np
import re

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0Z]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][8]))
#print(re.search('[a-z ]*prepared for business trips', df['Коммандировка'][8]))

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][0]))
#print(re.search('[a-z ]*prepared for [a-z]* business trips', df['Коммандировка'][0]))

df['Навыки'].iloc[0].split('\n')  # вот так?

work_list = df['Где работал?']
analytics_words = []
for work in work_list:
    # пока супер примитивное правило:
    if re.search('(SQL)|(Excel)|(Python)|(Tableau)|(Power BI)', work):
        analytics_words.append('found smt by rule (SQL)|(Excel)|(Python)|(Tableau)|(Power BI)')
    else:
        analytics_words.append('---')
df['Аналитические комбинации слов'] = analytics_words

**Перевод уровня образования:**

Определим уровень образования (высшее, ... — какие возможные значения?). Мы увидели разные стили написания. Составить в Markdown таблицу перевода.

In [32]:
z = list(np.unique(list(df['Уровень образование'])))

for y in z:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    print(y)
    
ys=[]
for y in df['Уровень образование']:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    ys.append(y)
df['Уровень оборазования нормализованный'] = ys

Высшее образование
Бакалавр
Магистр
Неоконченное высшее образование
Среднее специальное образование


**Сохранение результатов:**

In [33]:
list(df)[:6]

['Unnamed: 0',
 'Unnamed: 0.1',
 'Ссылка',
 'Название',
 'Коммандировка',
 'Опыт работы, лет']

**Теперь немного про карьерный рост:**

```
#df['Работа']
list(df)
```

для списка карьерного роста надо будет собрать список компаний где был и найти там совпадения, на текущих данных так сделать нельзя

In [43]:
driver = webdriver.Chrome('./chromedriver')

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_9946/2299280398.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Добавим еще один признак — карьерный рост: есть ли повторения в опыте работы? (Тоже наверх это)

In [44]:
repet = []

In [45]:
for i in range(df.shape[0]):
    driver.get(df.iloc[i]['Ссылка'])
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    w = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]/div[@class="bloko-text bloko-text_strong"][not(@data-qa="resume-block-experience-position")]//span')
    
    works = []
    for ww in w:
        works.append(ww.text)
    
    works_normilized = []
    for w in works:
        works_normilized.append(morph.normal_forms(w)[0])
    
    repet.append(len(set(works_normilized)) - len(works_normilized) < 0)

проверить, правда ли так много людей с повышениями:

In [46]:
repet[:3]

[True, False, False]

теперь для нормализованных названий найти повторения: `len(set(works)) - len(works) > 0` — и чекнуть адекватность такого определения

^ почти готово

**Для экономии времени — ведь этот признак был придуман позже**

In [48]:
df['Карьерный рост'] = repet

_куда-то надо сохранить этот новый датасет в приличное место (или вообще туда же, просто старую версию датасета переместить в backup)_:

In [ ]:
# dataset.save()

а также я давно хотел заняться топовостью обоазования

вообще там вроде все ок, тогда посмотрю вручную при разметке есть ли проблемы и почему, составлю список прямо

In [34]:
# df=df.drop(list(df)[0], axis=1)

Сохраним полученные результаты:

In [49]:
df.to_csv('resumes_features.csv')

### Что дальше? (Раньше было частью черновика)

In [ ]:
df = pd.read_csv('resumes_features.csv')

Осталось доделать справочник по образованию еще лучше, а также справочник по интересам и наличие аналитчиеских комбинаций слов (см черновик, что сделать)

**начинаем АД:**

ключевые совпадения см л34 она скинула

**Извлечение сущностей: более совершенные правила.** ВКР. Если город и коммандировка на английском то резюме точно на английском. Какие-то такие правила еще поискать. Посмотрю только в каком таком точном поле меньше всего пропусков. ВКР: позже вычленяю ещё сущности как мы обсуждали см prev call в 7.12. См github: допилить еще справочник образования и прочее, чтобы все в этом плане было идеально

В `scraping.ipynb` были собраны и подготовлены данные. Возможно, стоит еще доработать (в разделе "пост-обработка"):
- Словарь образования
- Справочник по интересам
- Аналитические комбинации слов

#### Предобработка полученных данных для анализа

**mBART перед построением токенов в "О себе"**

Описать как это работает. Вставить это перед построением эмбеддингов!!

https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta?text=%D0%97%D0%B0+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D0%B8+%D1%83%D1%87%D1%91%D0%B1%D1%8B%2C+%D1%8F+%D0%BE%D0%B2%D0%BB%D0%B0%D0%B4%D0%B5%D0%BB+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D1%8F%D0%BC%D0%B8%2C+%D0%BD%D0%B5%D0%BE%D0%B1%D1%85%D0%BE%D0%B4%D0%B8%D0%BC%D1%8B%D0%BC%D0%B8+%D0%B4%D0%BB%D1%8F+%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D0%B8+%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D0%B3%D0%BE+%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D1%8B%D1%85+%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B9+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%2C+%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0+%D0%B8+%D1%81%D0%B8%D0%BD%D1%82%D0%B5%D0%B7%D0%B0+%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D1%85+%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B9+%D0%B2+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D0%B0%D1%85+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D0%B8.%0A%D0%AF+%D0%BE%D0%B1%D0%BB%D0%B0%D0%B4%D0%B0%D1%8E+%D1%88%D0%B8%D1%80%D0%BE%D0%BA%D0%B8%D0%BC+%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D0%BE%D0%BC+%D0%B2+%D0%BF%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B8+BI+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8%2C+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8%2C+%D1%81%D0%B2%D1%8F%D0%B7%D1%8F%D1%85%2C+%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85+%D0%B2+Power+Qwery+%D0%B8+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D1%81+%D0%B1%D0%B0%D0%B7%D0%B0%D0%BC%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.%0A%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%B2%D0%BE%D0%B2%D0%B0%D0%BB+%D0%B2+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%D1%85+%D0%BF%D0%BE+%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B8+%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81-%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D0%B2+%D0%B8+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8.%0A%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BB+%D0%B2+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D1%8F%D1%85+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Power+BI%2C+Excel%2C+MS+Office%2C+VipPlanner.%0A%D0%97%D0%BD%D0%B0%D0%BA%D0%BE%D0%BC+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D1%8B%D0%BC%D0%B8+%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Jira%2C+kanban%2C+confluence.%0A%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B+%D0%B4%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4%D0%BE%D0%B2+%D0%BC%D0%BE%D0%B3%D1%83+%D0%B2%D1%8B%D1%81%D0%BB%D0%B0%D1%82%D1%8C+%D0%BB%D0%B8%D1%87%D0%BD%D0%BE.%0A%D0%91%D1%83%D0%B4%D1%83+%D1%80%D0%B0%D0%B4+%D0%B2+%D1%80%D0%B0%D0%BC%D0%BA%D0%B0%D1%85+%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B8%D1%82%D1%8C+%D0%BD%D0%B0+%D0%B2%D1%81%D0%B5+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D1%83%D1%8E%D1%89%D0%B8%D0%B5+%D0%92%D0%B0%D1%81+%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B.%0A%D0%97%D0%B0%D1%80%D0%B0%D0%BD%D0%B5%D0%B5+%D0%B1%D0%BB%D0%B0%D0%B3%D0%BE%D0%B4%D0%B0%D1%80%D1%8E+%D0%92%D0%B0%D1%81+%D0%B7%D0%B0+%D1%83%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D0%BE%D0%B5+%D0%BC%D0%BE%D0%B5%D0%B9+%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%B4%D0%B0%D1%82%D1%83%D1%80%D0%B5+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F.%0A%D0%92%D0%BB%D0%B0%D0%B4%D0%B5%D1%8E+%D0%BD%D0%B0%D0%B2%D1%8B%D0%BA%D0%BE%D0%BC+%22%D0%94%D0%B5%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F+%D0%BF%D0%B5%D1%80%D0%B5%

попробуем:

In [50]:
print(df['О себе'][14])

За время работы и учёбы, я овладел компетенциями, необходимыми для исследования, создания и успешного использования цифровых технологий моделирования, управления, анализа и синтеза системных решений в интересах создания сложной продукции.
Я обладаю широким функционалом в построении BI отчётности, моделировании, связях, корректировании данных в Power Qwery и работы с базами данных.
Участвовал в проектах по оптимизации и автоматизации бизнес-процессов и отчётности.
Работал в компаниях с такими программами как : Power BI, Excel, MS Office, VipPlanner.
Знаком с такими программными инструментами как : Jira, kanban, confluence.
Примеры дашбордов могу выслать лично.
Буду рад в рамках собеседования ответить на все интересующие Вас вопросы.
Заранее благодарю Вас за уделённое моей кандидатуре время.
Владею навыком "Деловая переписка".
Управление проектами PMBOK.

Строил аналитические дашборды для портала HR-дашборд в компании МТС.
Сейчас руковожу проектом по внедрению аналитических дашбордов по 

In [51]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [52]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

^ ок, снизу ... просто тест

In [87]:
def summarize(article_text):
    input_ids = tokenizer(
        [article_text],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

Как работает функция:

In [88]:
df['О себе'].iloc[0]

'В последние годы проходила обучение без возможности работать.\nGPA 8,5.\nКурсовая работа по теме "Создание и разработка базы данных аэропорта" с оценкой 9.\nВ данный момент принимаю участие в учебном проекте "Внедрение системы для автоматизации бюджетирования и консолидации отчетности аэропорта", где получаю опыт в интервьюировании заказчика, описании бизнес-задач, формализации требований, формировании устава проекта и тд.'

In [89]:
summarize(df['О себе'].iloc[0])

'Гражданин России, проживающий в Санкт-Петербурге, занимаюсь созданием системы для автоматизации бюджетирования и консолидации отчетности аэропорта. На данный момент работаю в учебном проекте, где получаю опыт в интервьюировании заказчика, описании бизнес-задач, формализации требований, формировании устава проекта и тд.'

In [124]:
summarizations = []
for i in range(df.shape[0]):
    if df['О себе'].iloc[i] != '---' and df['is_eng?'].iloc[i] == 'RU' and (not pd.isna(df['О себе'].iloc[i])):
        text_to_summarize = df['О себе'].iloc[i]
        summarizations.append(summarize(text_to_summarize))
    else:
        summarizations.append(df['О себе'].iloc[i])
    
    if len(summarizations) % 5 == 0:
        print(len(summarizations), '/', df.shape[0])

KeyboardInterrupt: 

In [121]:
df['О себе, summarized'] = summarizations

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_9946/1096740388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['О себе, summarized'] = summarizations


In [123]:
df.to_csv('resumes_features.csv')

Дальше: article_text = `df['О себе'][0]`; мб надо будет по частям обрабатывать ибо это будет долго и затратно по памяти чтобы ничего не упало и не испортило всю работу

Итак, мы извлекли сущности и привели данные к деиному формату. Теперь займемся дальнейшей предобработкой (этапом): токенизация.

Мы будем осущесвлять ее с помощью следующего алгоритма. Берем ...

**Токенизация раздела "О себе" на 1-граммы:**

In [125]:
df = pd.read_csv('resumes_features.csv')

In [126]:
import nltk
# nltk.download('stopwords')

import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    # print(doc)
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru and morph.normal_forms(token)[0] not in stopwords_ru:  # ВОТ откуда там стоп-слвоа — В not in stopwords_ru -> normal(В) = в
            # print(token)
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

df2 = pd.DataFrame(df['О себе'])

for col_ind in df2.index:
    col = df2.iloc[col_ind]['О себе']
    if col:
        df2.iloc[col_ind]['О себе'] = lemmatize(str(col))
        #print(lemmatize(str(col)))
        
data = df2['О себе']

from collections import defaultdict
word_freq = defaultdict(int)
for tokens in data.iloc[:]:
    if tokens:
        for token in tokens:
            word_freq[token] += 1

In [127]:
df2.sample(3)

,О себе
15,None
55,None
24,"[курс, экономический, факультет, ниу, вшэ, сре..."


^ Стоп-слова убраны, все в нормальной форме => их не будет и дальше.

На 2-граммы:

В соответствии с задачей работы, мы разобьем на 3-граммы. Но сначала на 2-граммы. Для этого будем перебирать все 1-граммы (изначальные токены) и соединять.

In [128]:
from nltk.util import ngrams
from nltk import word_tokenize 

tokens2 = []

for x in df2['О себе']:
    if x is not None:
        bigrams = []
        for bg in list(ngrams(x, 2)):
            bigrams.append(' '.join(bg))
        tokens2.append(bigrams)
    else:
        tokens2.append(None)
        
df2['О себе 2'] = tokens2

In [129]:
df2.sample(3)

,О себе,О себе 2
46,"[уметь, работать, sql, запрос, написание, алго...","[уметь работать, работать sql, sql запрос, зап..."
1,None,None
18,"[навык, умение, business, intelligence, сбор, ...","[навык умение, умение business, business intel..."


На 3-граммы:

Все аналогично.

In [130]:
from nltk.util import ngrams
from nltk import word_tokenize 

tokens3 = []

for x in df2['О себе']:
    if x is not None:
        trigrams = []
        for tg in list(ngrams(x, 3)):
            trigrams.append(' '.join(tg))
        tokens3.append(trigrams)
    else:
        tokens3.append(None)
        
df2['О себе 3'] = tokens3

In [131]:
df2.sample(3)

,О себе,О себе 2,О себе 3
37,"[высокий, степень, ответственность, целеустрем...","[высокий степень, степень ответственность, отв...","[высокий степень ответственность, степень отве..."
9,"[свободный, владение, компьютер, ms, office, e...","[свободный владение, владение компьютер, компь...","[свободный владение компьютер, владение компью..."
5,None,None,None


Строим эмбеддинги с помощью среднего word2vec-вектора токенов (но сначала надо заденойзить `О себе` и вообще использовать все данные!!):

In [132]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

Избавились от пропусков:

In [133]:
df2.loc[df2['О себе 3'].isna(), 'О себе 3'] = '-'

In [134]:
df2[df2['О себе 3'].isna()]['О себе 3']

Series([], Name: О себе 3, dtype: object)

In [135]:
model = Word2Vec(sentences=df2['О себе 3'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [136]:
model = Word2Vec.load("word2vec.model")

In [137]:
model

In [138]:
model.train(df2['О себе 3'], total_examples=len(df2['О себе 3']), epochs=1)

(3373, 3379)

^ наверное лучше еще привести все токены в тексте к норм. форме?

In [139]:
df2['О себе 3']

0     [последний год проходить, год проходить обучен...
1                                                     -
2     [иметь экономический образование, экономически...
3     [иметься опыт создание, опыт создание android,...
4     [уверенный пользователь желание, пользователь ...
                            ...                        
62    [последний год проходить, год проходить обучен...
63    [владеть инструментарий обработка, инструмента...
64    [победитель стипендиальный конкурс, стипендиал...
65    [добрый день звать, день звать константин, зва...
66    [опыт web разработка, web разработка использов...
Name: О себе 3, Length: 67, dtype: object

In [140]:
vector = model.wv['последний год проходить']  # get numpy vector of a word
sims = model.wv.most_similar('последний год проходить', topn=10)  # get other similar words

In [141]:
sims

[('который первый год', 0.3447602093219757),
 ('помимо университетский дисциплина', 0.32323339581489563),
 ('sf education осень', 0.3145250678062439),
 ('проект поставщик luxoft', 0.2966373860836029),
 ('аналитик компания внедрять', 0.28306668996810913),
 ('работа аналитический система', 0.2808486223220825),
 ('позволять осуществлять проект', 0.27972176671028137),
 ('искусство лето закончить', 0.27328839898109436),
 ('global management challenge', 0.27294981479644775),
 ('закончить курс профессиональный', 0.2656009793281555)]

In [142]:
vector

array([ 4.2191329e-03,  3.8639535e-04, -8.7988045e-04, -3.2099015e-03,
        4.9834154e-03, -9.1659417e-03,  5.0880262e-03,  5.7413573e-03,
       -2.4297498e-03, -4.2949175e-03, -7.4386951e-03, -6.4669657e-03,
       -5.1392550e-03,  3.1431972e-03, -4.8158299e-03, -1.1919099e-03,
       -8.9776153e-03, -3.5633948e-03, -7.2716450e-04, -7.9507641e-03,
        3.1933037e-03,  9.7603686e-03,  4.3936232e-03,  1.8984299e-03,
       -4.8812316e-03, -1.0939811e-03,  2.2118050e-03,  1.7161242e-03,
        8.7872315e-03, -9.4179786e-04,  6.9992980e-03,  5.7946197e-03,
        3.0359959e-03,  7.3704869e-03, -4.9583632e-03, -3.9206040e-03,
        6.9923629e-03,  8.5658487e-03,  3.1426456e-03, -4.1517466e-03,
        7.6696882e-03,  2.7365047e-03, -2.2905457e-04,  8.8905348e-03,
        4.2693964e-03,  3.9646570e-03, -8.5953310e-05, -8.9292033e-03,
       -3.1199013e-03,  2.4929452e-03, -5.6297285e-03,  7.1631474e-03,
        7.4526034e-03, -8.9817978e-03, -5.4336186e-03, -6.0272841e-03,
      

Итак, мы построили эмбеддинги. Теперь кластеризуем. Для этого сначала строим новую табличку:

In [143]:
model.wv

In [144]:
our_data = pd.DataFrame(df2['О себе 3'])

In [145]:
our_data['emb'] = 0

In [146]:
our_data

,О себе 3,emb
0,"[последний год проходить, год проходить обучен...",0
1,-,0
2,"[иметь экономический образование, экономически...",0
3,"[иметься опыт создание, опыт создание android,...",0
4,"[уверенный пользователь желание, пользователь ...",0
...,...,...
62,"[последний год проходить, год проходить обучен...",0
63,"[владеть инструментарий обработка, инструмента...",0
64,"[победитель стипендиальный конкурс, стипендиал...",0
65,"[добрый день звать, день звать константин, зва...",0


In [147]:
embs = []
for x in our_data['О себе 3']:
    cur_embs = []
    for y in x:
        cur_embs.append(model.wv[y])
    embs.append(cur_embs)

In [148]:
our_data['emb'] = embs

теперь надо построить `flat_emb` — по векторам эмбеддингов построить единый вектор эмбеддингов.

2ая попытка:

In [149]:
# буду использовать среднее

embs_flattened = []
for x in our_data['emb']:
    x_flat = np.array(x[0])
    for z in x[1:]:
        x_flat += np.array(z)
    embs_flattened.append(x_flat / len(x))

In [150]:
embs_flattened[4].size

100

почему это работает: Для ВКР и IR одновременно: https://stats.stackexchange.com/questions/318882/what-does-average-of-word2vec-vector-mean

1ая попытка:

```
embs_flattened = []
for x in our_data['emb']:
    x_flat = []
    for z in x:
        x_flat += list(z)
    embs_flattened.append(x_flat)
```

после поаытки:

In [151]:
our_data['flat_emb'] = embs_flattened

в чем проблема с `flat_emb` как эмбеддингами??? именно поэтому не работает kmeans.

Итак, у меня для каждого токена в тексте есть эмбеддинг. Как из этого получить что-то адекватное?

In [152]:
our_data['flat_emb'][0]

array([ 2.95168342e-04, -1.54395448e-03, -1.38613814e-03, -9.72507230e-04,
        1.15804060e-03, -1.65132154e-03, -1.30809122e-03,  1.68300045e-04,
        6.33979682e-04, -5.52685000e-04, -3.46425048e-04, -1.96835026e-03,
       -4.65927005e-04, -1.04350016e-04, -2.10811268e-04, -5.57828054e-04,
       -1.20974914e-03, -8.23530194e-04, -9.68457549e-04, -1.87526247e-03,
       -1.47328584e-03, -2.93521851e-04,  1.04553299e-04,  1.60692690e-03,
        8.10740967e-05,  1.31653636e-04,  7.93572035e-05,  4.58291688e-05,
       -2.51170772e-04,  1.39464828e-04,  8.84371344e-04, -4.17852541e-04,
       -1.78421120e-04, -2.19719048e-04, -3.30868410e-04, -6.14361605e-04,
        1.18277618e-04,  2.18484172e-04,  1.57382136e-04,  4.52921871e-04,
        1.20168820e-03,  1.97265137e-04,  6.45224354e-04, -3.41328676e-04,
        3.97570606e-04, -1.13717723e-03,  1.53014145e-03, -1.96184446e-05,
        2.31549959e-03, -2.36989319e-04,  1.12825153e-04, -2.76852574e-04,
       -8.87530099e-04, -

In [153]:
our_data['flat_emb'].dropna()

0     [0.00029516834, -0.0015439545, -0.0013861381, ...
1     [-0.0086196875, 0.003665738, 0.0051898835, 0.0...
2     [-0.0011685997, 0.0012998827, -0.0025936777, 0...
3     [-0.0004074424, 0.0005873837, 0.0008515791, 0....
4     [-0.00037161107, 0.00023680142, -0.00016213297...
                            ...                        
62    [-0.00049789756, -0.00054419227, -0.0025611392...
63    [-0.0021155155, -0.0005143842, 0.0004572586, -...
64    [7.965116e-06, 2.841573e-05, 0.00020546571, 0....
65    [0.0008847946, 0.0004886352, 0.00014909195, 0....
66    [-6.5130975e-05, -0.00063547277, -0.000924198,...
Name: flat_emb, Length: 67, dtype: object

In [154]:
our_data['flat_emb'] * 2

0     [0.0005903367, -0.003087909, -0.0027722763, -0...
1     [-0.017239375, 0.007331476, 0.010379767, 0.011...
2     [-0.0023371994, 0.0025997653, -0.0051873554, 0...
3     [-0.0008148848, 0.0011747674, 0.0017031583, 0....
4     [-0.00074322213, 0.00047360285, -0.00032426594...
                            ...                        
62    [-0.0009957951, -0.0010883845, -0.0051222784, ...
63    [-0.004231031, -0.0010287684, 0.0009145172, -0...
64    [1.5930233e-05, 5.683146e-05, 0.00041093142, 0...
65    [0.0017695891, 0.0009772704, 0.0002981839, 0.0...
66    [-0.00013026195, -0.0012709455, -0.001848396, ...
Name: flat_emb, Length: 67, dtype: object

In [155]:
our_data['flat_emb']

0     [0.00029516834, -0.0015439545, -0.0013861381, ...
1     [-0.0086196875, 0.003665738, 0.0051898835, 0.0...
2     [-0.0011685997, 0.0012998827, -0.0025936777, 0...
3     [-0.0004074424, 0.0005873837, 0.0008515791, 0....
4     [-0.00037161107, 0.00023680142, -0.00016213297...
                            ...                        
62    [-0.00049789756, -0.00054419227, -0.0025611392...
63    [-0.0021155155, -0.0005143842, 0.0004572586, -...
64    [7.965116e-06, 2.841573e-05, 0.00020546571, 0....
65    [0.0008847946, 0.0004886352, 0.00014909195, 0....
66    [-6.5130975e-05, -0.00063547277, -0.000924198,...
Name: flat_emb, Length: 67, dtype: object

In [156]:
X = list(our_data['flat_emb'].dropna())

In [157]:
X = np.asarray(X)

In [158]:
X

array([[ 2.9516834e-04, -1.5439545e-03, -1.3861381e-03, ...,
        -1.3355449e-03, -9.9455914e-04, -7.4276538e-04],
       [-8.6196875e-03,  3.6657380e-03,  5.1898835e-03, ...,
        -2.3915148e-03, -9.5100952e-03,  4.5058774e-03],
       [-1.1685997e-03,  1.2998827e-03, -2.5936777e-03, ...,
        -7.4811236e-05,  6.1857427e-04, -3.5170032e-04],
       ...,
       [ 7.9651163e-06,  2.8415730e-05,  2.0546571e-04, ...,
        -1.4128849e-04, -6.5393321e-04,  1.0128486e-03],
       [ 8.8479457e-04,  4.8863521e-04,  1.4909195e-04, ...,
        -8.6324412e-04, -1.2545983e-03,  7.0064142e-04],
       [-6.5130975e-05, -6.3547277e-04, -9.2419802e-04, ...,
         6.9236971e-04, -5.9665169e-04, -7.2573387e-04]], dtype=float32)

In [159]:
for x in X:

IndentationError: expected an indented block (1027233577.py, line 1)

In [211]:
X = np.array([np.array([1, 2, 3]), np.array([2, 3, 4]), np.array([5, 6, 7])])

In [161]:
embs_flattened[0]

array([ 2.95168342e-04, -1.54395448e-03, -1.38613814e-03, -9.72507230e-04,
        1.15804060e-03, -1.65132154e-03, -1.30809122e-03,  1.68300045e-04,
        6.33979682e-04, -5.52685000e-04, -3.46425048e-04, -1.96835026e-03,
       -4.65927005e-04, -1.04350016e-04, -2.10811268e-04, -5.57828054e-04,
       -1.20974914e-03, -8.23530194e-04, -9.68457549e-04, -1.87526247e-03,
       -1.47328584e-03, -2.93521851e-04,  1.04553299e-04,  1.60692690e-03,
        8.10740967e-05,  1.31653636e-04,  7.93572035e-05,  4.58291688e-05,
       -2.51170772e-04,  1.39464828e-04,  8.84371344e-04, -4.17852541e-04,
       -1.78421120e-04, -2.19719048e-04, -3.30868410e-04, -6.14361605e-04,
        1.18277618e-04,  2.18484172e-04,  1.57382136e-04,  4.52921871e-04,
        1.20168820e-03,  1.97265137e-04,  6.45224354e-04, -3.41328676e-04,
        3.97570606e-04, -1.13717723e-03,  1.53014145e-03, -1.96184446e-05,
        2.31549959e-03, -2.36989319e-04,  1.12825153e-04, -2.76852574e-04,
       -8.87530099e-04, -

In [162]:
len(embs_flattened[0])

100

In [233]:
X = embs_flattened

In [50]:
for x in our_data['emb'][0]:
    print(len(x))

100
100
100
100
100


In [163]:
our_data['emb']

0     [[0.004219133, 0.00038639535, -0.00087988045, ...
1     [[-0.0086196875, 0.003665738, 0.0051898835, 0....
2     [[-0.005774332, -0.0029801906, 0.006394581, 0....
3     [[-0.008720077, 0.009953186, -0.0023454358, 0....
4     [[0.0062332065, -0.0016407053, 0.005968472, -2...
                            ...                        
62    [[0.004219133, 0.00038639535, -0.00087988045, ...
63    [[0.0021670053, 0.0050119366, 0.006372746, -0....
64    [[0.0063795494, 0.0014176572, 0.006802543, 0.0...
65    [[-0.0007966944, -0.0015513458, 0.00490522, -0...
66    [[-0.006899624, 0.009475157, 0.009700476, -0.0...
Name: emb, Length: 67, dtype: object

**ЭТО ТОЖЕ НАДО БУДЕТ СОХРАНИТЬ В RESUMES_FEATURES** в уже более аккуратном окончательном виде

Но непонятно, как это сделать :(

In [166]:
df['О себе, tokens'] = our_data['emb']

In [167]:
our_data['emb']

0     [[0.004219133, 0.00038639535, -0.00087988045, ...
1     [[-0.0086196875, 0.003665738, 0.0051898835, 0....
2     [[-0.005774332, -0.0029801906, 0.006394581, 0....
3     [[-0.008720077, 0.009953186, -0.0023454358, 0....
4     [[0.0062332065, -0.0016407053, 0.005968472, -2...
                            ...                        
62    [[0.004219133, 0.00038639535, -0.00087988045, ...
63    [[0.0021670053, 0.0050119366, 0.006372746, -0....
64    [[0.0063795494, 0.0014176572, 0.006802543, 0.0...
65    [[-0.0007966944, -0.0015513458, 0.00490522, -0...
66    [[-0.006899624, 0.009475157, 0.009700476, -0.0...
Name: emb, Length: 67, dtype: object

In [205]:
our_data['flat_emb_avg'] = ''
flat_emb_avg = []

In [206]:
for x in our_data['flat_emb']:
    flat_emb_avg.append(np.mean(x))
our_data['flat_emb_avg'] = flat_emb_avg
our_data['flat_emb_avg']

0    -0.000129
1     0.000681
2    -0.000146
3     0.000049
4    -0.000058
        ...   
62   -0.000504
63   -0.000174
64    0.000066
65   -0.000009
66   -0.000126
Name: flat_emb_avg, Length: 67, dtype: float64

In [207]:
np.mean(our_data['flat_emb'][0])

-0.0001285661

вот это НН:

In [208]:
df['О себе, avg_tokens'] = our_data['flat_emb_avg']

In [209]:
df['О себе, 3-grams'] = our_data['О себе 3']

In [210]:
df.to_csv('resumes_features.csv')

Вот эту штуку мы загружаем в алгоритм кластеризации, надо тоже ее сохранить в таблицу: 

In [308]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000295,-0.001544,-0.001386,-0.000973,0.001158,-0.001651,-0.001308,0.000168,0.000634,-0.000553,...,-0.001833,-0.000138,0.000944,0.000743,-0.000198,0.000212,1.081802e-03,-0.001336,-0.000995,-0.000743
1,-0.008620,0.003666,0.005190,0.005742,0.007467,-0.006168,0.001106,0.006047,-0.002840,-0.006174,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,5.346682e-03,-0.002392,-0.009510,0.004506
2,-0.001169,0.001300,-0.002594,0.000370,0.000996,-0.003299,0.000349,0.000345,-0.000463,-0.000644,...,-0.002032,-0.000797,0.000857,0.001121,0.000713,0.001256,2.744928e-07,-0.000075,0.000619,-0.000352
3,-0.000407,0.000587,0.000852,0.000703,-0.000880,0.000500,-0.000126,0.000755,0.000299,0.000519,...,0.001694,0.001077,0.001228,-0.000574,0.000283,-0.001303,-1.152844e-04,0.000962,0.000771,-0.000796
4,-0.000372,0.000237,-0.000162,-0.000195,-0.000445,0.000184,-0.000011,-0.000585,-0.000691,-0.000110,...,-0.000880,0.000244,0.000685,-0.000700,-0.000378,0.000316,3.890419e-04,-0.000418,-0.000869,-0.000259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,-0.000498,-0.000544,-0.002561,0.001359,0.000329,-0.005534,-0.001044,0.002980,-0.003545,-0.007325,...,0.001095,-0.001868,-0.000197,0.004056,-0.000613,0.004742,3.581134e-03,-0.001931,-0.005528,0.002591
63,-0.002116,-0.000514,0.000457,-0.000912,-0.000279,-0.001034,0.000015,-0.002101,0.000427,-0.000185,...,-0.001381,-0.000462,-0.000876,-0.002568,0.000448,-0.003808,4.142171e-04,0.002105,-0.002256,-0.000658
64,0.000008,0.000028,0.000205,0.000259,0.000265,0.000270,0.000110,0.000886,0.000519,0.000684,...,-0.000178,-0.000481,0.000166,0.000323,-0.000305,-0.000398,6.580245e-04,-0.000141,-0.000654,0.001013
65,0.000885,0.000489,0.000149,0.000421,-0.001353,0.000044,0.001128,0.000413,-0.000123,-0.001166,...,-0.001279,-0.000413,0.000310,0.000159,0.000438,-0.000380,9.543705e-04,-0.000863,-0.001255,0.000701


Займусь этим позже, когда приступлю к анализу данных.